# Wörterbuchbasierte Sentimentanalyse

- dies ist ein Besipiel für die wörterbuchbasierte Sentiment-Analyse nach dem Vorbild des DiSpecs-Projekts (Koncar, Philipp; Geiger, Bernhard; Glatz, Christina; Hobisch, Elisabeth, Sarić, Sanja; Scholger, Martina; Völkl, Yvonne; Helić, Denis (2022): A Sentiment Analysis Tool Chain for 18th Century Periodicals. In: Fabrikation von Erkenntnis: Experimente in den Digital Humanities https://doi.org/10.26298/EZPG-WK34 [Stand: 12.02.2023])
- der Code wurde angepasst und erweitert

- als Erstes müssen folgende Module importiert werden:

In [ ]:
import os
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML

## Dataframe-Erstellung

- hier werden die Namen der Dateien extrachiert und an eine Liste gegeben:

In [ ]:
pfad = r"pfad\zum\ordner"
os.chdir(pfad)

names=[]
for datei in os.listdir(pfad): 
    f = os.path.join(pfad, datei) 
    if os.path.isfile(f): 
        name, endung=datei.split('.')
        titel = names.append(name)

- der Inhalt der Dateien wird eingelesen und an eine Liste gegeben:

In [ ]:
briefe=[]
for datei in os.listdir(pfad): 
    f = os.path.join(pfad, datei) 
    if os.path.isfile(f): 
        with open (f, encoding = "UTF-8") as fh:
            brief = fh.read()
            briefe.append(brief.replace('\n\n', '\n'))

- es wird eine Tabelle erstellt, die drei Spalten haben soll (briefe, name und sentiment):

In [ ]:
df = pd.DataFrame(np.array(briefe), columns=['briefe'])

In [ ]:
df.insert(loc=0, column='name', value=names)

- Wörterbücher, d.h. eine Liste mit negativen und eine Liste mit positiven Wörtern, werden eingelesen (Achtung: bei der Sentiment-Berechnung mit Negation müssen entsprechende Negationen aus dem Wörterbuch entfernt werden):

In [ ]:
with open(r"pfad\zur\datei1", encoding='utf-8') as fh:
    sentiment_dict_neg = fh.read().lower().splitlines()
with open(r"pfad\zur\datei2", encoding='utf-8') as fh:
    sentiment_dict_pos = fh.read().lower().splitlines()
print("loaded {} negative words".format(len(sentiment_dict_neg)))
print("loaded {} positive words".format(len(sentiment_dict_pos)))

- Funktion zur Berechung des Sentiments wird eingeführt:

In [ ]:
def compute_sentiment(text):
    tokens = nltk.word_tokenize(text)
    tokens = [t.lower() for t in tokens]
    num_negative = 0
    num_positive = 0
    for nw in sentiment_dict_neg:
        num_negative += tokens.count(nw.lower())
    for pw in sentiment_dict_pos:
        num_positive += tokens.count(pw.lower())
    try:
        sentiment_score = (num_positive - num_negative) / (num_positive + num_negative)
    except ZeroDivisionError:
        sentiment_score = 0
    return sentiment_score

- Funktion zur Berechnung des Sentiments, die auch die Negationen "nicht", "nichts", "kein" (in allen Formen) und "ohne" im Text berücksichtigt:

In [ ]:
def compute_sentiment_score(text):
    tokens = nltk.word_tokenize(text) 
    tokens = [t.lower() for t in tokens] 
    num_negative = 0     
    num_positive = 0     
    for i, token in enumerate(tokens): 
        
        if token in sentiment_dict_pos:      
            if i>0 and tokens[i-1] in ['nicht', 'nichts', 'kein', 'keine', 'keiner', 'keines', 'keinem', 'keinen', 'ohne']:  
                num_negative += 1                   
            else:                                   
                num_positive += 1                   
        elif token in sentiment_dict_neg:          
            if i>0 and tokens[i-1] in ['nicht', 'nichts', 'kein', 'keine', 'keiner', 'keines', 'keinem', 'keinen', 'ohne']:      
                num_positive += 1                 
            else:                                   
                num_negative += 1                  
    try:                                            
        sentiment_score = (num_positive - num_negative) / (num_positive + num_negative) 
    except ZeroDivisionError:                     
        sentiment_score = 0
    return sentiment_score

- die Berechnung wird durchgeführt und die Ergebnisse werden an die Tabelle weitergegeben:

In [ ]:
df["sentiment"] = df["briefe"].apply(compute_sentiment)

- die Tabelle wird gespeichert:

In [ ]:
df.to_pickle("dateiname.p")

- die Statistik aus der Tabelle wird ausgegeben:

In [ ]:
df["sentiment"].describe()

- Pfad zum Ordner, in dem die Visualisierung gespeichert werden soll, und das Format, in dem gespeichert wird, werden angegeben: 

In [ ]:
plot_results_dir = r"pfad\zum\plotorder\\"
plot_file_format= "png"

## Attribute Plots

- das Attribut, das analysiert wird, wird angegeben

In [ ]:
attribute = "name"

### Bar Plot

- Erstellung eines vertikalen Balkendiagramms:

In [ ]:
ax = df.groupby(attribute)["sentiment"].mean().plot(kind="bar", ylabel="Sentiment Score")
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.savefig("{}{}_bar_plot.{}".format(plot_results_dir, attribute, plot_file_format))
plt.show()
plt.close()

- Erstellung eines horizontalen Diagramms für große Datensätze:

In [ ]:
df.groupby(attribute)["sentiment"].mean().plot(kind="barh")
fig = plt.gcf()
fig.set_size_inches(18.5, 110.5)
plt.title("briefgruppe")
plt.ylabel("names")
plt.xlabel("sentiment")

### Sentiment Words Highlighting

- Visualisierung einzelner Texte - positive und negative Wörter werden fett und farblich (positive grün, negative rot) markiert dargestellt:

In [ ]:
text_to_print = df.loc[briefnr., "briefe"] # z.B. text_to_print = df.loc[50, "briefe"]
for nw in sentiment_dict_neg:
    if re.search(r'\b'+re.escape(nw)+r'\b', text_to_print, re.IGNORECASE) and nw not in ["span", "style", "color", "font", "size"]:
        pattern = re.compile(r'(\b'+re.escape(nw)+r'\b)', re.IGNORECASE)
        text_to_print = pattern.sub(r"<span style='color:#E74C3C; font-size:20pt'><b>\1</b></span>", text_to_print)
        
for pw in sentiment_dict_pos:
    if re.search(r'\b'+re.escape(pw)+r'\b', text_to_print, re.IGNORECASE) and pw not in ["span", "style", "color", "font", "size"]:
        pattern = re.compile(r'(\b'+re.escape(pw)+r'\b)', re.IGNORECASE)
        text_to_print = pattern.sub(r"<span style='color:#27AE60; font-size:20pt'><b>\1</b></span>", text_to_print)

HTML(text_to_print)

- Visualisierung einzelner Texte - Negation, positive und negative Wörter werden fett und farblich (Negation blau, positive Wörter grün, negative rot) markiert dargestellt:

In [ ]:
text_to_print = df.loc[briefnr., "briefe"] # z.B. text_to_print = df.loc[50, "briefe"]
for neg in ['nicht', 'nichts', 'kein', 'keine', 'keiner', 'keines', 'keinem', 'keinen', 'ohne']:
    if neg.lower() in text_to_print.lower():
        text_to_print = re.sub(r"\b{}\b".format(neg), r"<span style='color:#2a31b0; font-size:20pt'><b>{}</b></span>".format(neg), text_to_print)
for nw in sentiment_dict_neg:
    if nw.lower() in text_to_print.lower() and nw not in ["span", "style", "color", "font", "size"]:
        text_to_print = re.sub(r"\b{}\b".format(nw), r"<span style='color:#E74C3C; font-size:20pt'><b>{}</b></span>".format(nw), text_to_print)
        
for pw in sentiment_dict_pos:
    if pw.lower() in text_to_print.lower() and pw not in ["span", "style", "color", "font", "size"]:
        text_to_print = re.sub(r"\b{}\b".format(pw), r"<span style='color:#27AE60; font-size:20pt'><b>{}</b></span>".format(pw), text_to_print)

HTML(text_to_print)

# Statistik

- Beispielcodes für Balkendiagramme, die einen Vergleich der Durchschnittswerte darstellen:

In [ ]:
#Vergleich der durchschnittlichen Setniment-Werte pro Textgruppe 
plt.rc('font', size=18)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

gruppe = ['Gruppe1', 'Gruppe2', 'Gruppe3', 'Gruppe4', ...]
sentiment_durchschnitt = [Wert1, Wert2, Wert3, Wert4, ...]
ax.bar(gruppe,sentiment_durchschnitt)
ax.set_xlabel('Briefgruppe')
ax.set_ylabel('Sentiment-Durchschnitt')
ax.set_title('Durchschnittlicher Sentiment-Wert pro Gruppe')
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
# Darstellung der Durchschnittwerte in einzelnen Gruppen und nach verschiedenen Krieterien (hier Wörterbüchern) in verschiedenen Farben

data=[["Briefgruppe1", Wert1, Wert2, Wert3, Wert4, ...],
      ["Briefgruppe2", Wert1, Wert2, Wert3, Wert4, ...],
      ["Briefgruppe3", Wert1, Wert2, Wert3, Wert4, ...],
      ["Briefgruppe4", Wert1, Wert2, Wert3, Wert4, ...],
      ["Briefgruppe5", Wert1, Wert2, Wert3, Wert4, ...],
      ...
      ]
df=pd.DataFrame(data,columns=["Gruppe","Wörterbuch1","Wörterbuch2","Wörterbuch3", "Wörterbuch4", ...])

ax = df.plot(x="Gruppe", y=["Wörterbuch1","Wörterbuch2","Wörterbuch3", "Wörterbuch4", ...], kind="bar",figsize=(18,10))
ax.set_ylabel("Sentiment-Durchschnitt")
plt.show()

In [ ]:
# horizontales Balkendiagramm für große Briefgruppen
attribute = "name"
df_fnd.groupby(attribute)["sentiment"].mean().plot(kind="barh")
fig = plt.gcf()
fig.set_size_inches(40, 300)
plt.rc('font', size=18)
plt.title("briefgruppe")
plt.ylabel("names")
plt.xlabel("sentiment")
plt.savefig("{}{}_bar_plot.{}".format(plot_results_dir, attribute, plot_file_format))